### UC Berkeley, MICS, W202-Cryptography
### Week 06 Breakout 2
### Diffie Hellman Elliptic Curve Key Exchange


The Diffie Hellman Elliptic Curve Key Exchange is based on the computational intractibility of the Discrete Logarithm Elliptic Curve Problem for a large prime number p.

You will find it very similar to the Diffie Hellman Key Exchange based on the Discrete Logarithm Problem we previously studied. The main difference is that instead of exponents, we will use scalar multipliers for points on an elliptic curve prime curve.

Since Diffie Hellman Elliptic Curve Key Exchange is so similar to Diffie Hellman Key Exchange, it should be no surprise that it is also subject to man in the middle attacks if there is no proper authentication of Alice and Bob.

The Diffie Hellman Elliptic Curve Key Exchange algorithm works as follows (using the names and notation from our lectures):

* Publically known: An Elliptic Curve prime curve E in modulo prime p.  A, B, p, P are publically known.

* Alice generates a large random number of similar bit size to p that is < p.  The number can be odd or even, prime or composite.  This is Alice's scalar multiplier, which she must keep secret!

* Alice calculates alice_scalar * P (mod p) and sends this point to Bob.

* Bob should validate the point he receives from Alice to guard against an invalid curve attack.

* Likewise, Bob generates a scalar multiplier, calculates bob_scalar * P (mod p) and sends this point to Alice.  Bob of course, keeps his scalar multiplier secret!

* Alice should validate the point she receives from Bob to guard against an invalid curve attack.

* Alice can calculate the key (point) by multiplying the point Bob sent her by her secret scalar multiplier in modulo p.

* Likewise, Bob can calculate the key (point) by multiplying the point Alice sent him by his secret scalar multiplier in modulo p.

* Alice and Bob now have the same key (point).  Note that neither Bob or Alice picked the key.  Note also that they key is a point on an elliptic curve, essentially two large integers.

* Since the generated key is a point on an elliptic curve, and we typically want to use it as a key for AES256 symmetrical encryption with a 256 bit key, it's common to concatenate the x and y values of the point and take a SHA256 hash.

In [1]:
import hashlib
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
def my_random_k(p):
    "Given prime p, determine it's bit size, generate a random number 1 bit less than p's size"
    
    upper_limit = (p-2)
    lower_limit = ( (p-2) // 2)
    
    return Integer(randint(lower_limit, upper_limit))

#### We start with a publically known Elliptic Curve Prime Curve in modulo p.  A, B, p, P are publically known.

In [4]:
# FIPS Curve P-256

p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff

A = -3
B = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b

P_x = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
P_y = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5

E = EllipticCurve(GF(p), [A,B])

P = E(P_x,P_y)

In [5]:
my_print_number("A", A)


 A 

decimal: -3 

number of digits: 1 

hex: -3 

number of bits: 2 



In [6]:
my_print_number("B", B)


 B 

decimal: 41,058,363,725,152,142,129,326,129,780,047,268,409,114,441,015,993,725,554,835,256,314,039,467,401,291 

number of digits: 77 

hex: 5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b 

number of bits: 255 



In [7]:
my_print_number("p", p)


 p 

decimal: 115,792,089,210,356,248,762,697,446,949,407,573,530,086,143,415,290,314,195,533,631,308,867,097,853,951 

number of digits: 78 

hex: ffffffff00000001000000000000000000000000ffffffffffffffffffffffff 

number of bits: 256 



In [8]:
my_print_number("Px", P_x)


 Px 

decimal: 48,439,561,293,906,451,759,052,585,252,797,914,202,762,949,526,041,747,995,844,080,717,082,404,635,286 

number of digits: 77 

hex: 6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296 

number of bits: 255 



In [9]:
my_print_number("Py", P_y)


 Py 

decimal: 36,134,250,956,749,795,798,585,127,919,587,881,956,611,106,672,985,015,071,877,198,253,568,414,405,109 

number of digits: 77 

hex: 4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5 

number of bits: 255 



#### Alice generates a large random number of similar bit size to p that is < p. The number can be odd or even, prime or composite. This is Alice's scalar multiplier, which she must keep secret!

In [10]:
alice_scalar = my_random_k(p)

In [11]:
my_print_number("Alice's secret scalar multiplier", alice_scalar)


 Alice's secret scalar multiplier 

decimal: 88,622,257,418,936,847,494,394,043,435,200,814,574,158,786,754,556,962,751,357,760,305,565,585,097,072 

number of digits: 77 

hex: c3ee6b8f0fadc34f5fae5c626889fc6d86c86abef2dbec868942a6a61d47c170 

number of bits: 256 



#### Alice calculates alice_scalar * P and sends this point to Bob.

In [12]:
alice_sends = alice_scalar * P

In [13]:
my_print_number("Alice sends Bob - x value of point", Integer(alice_sends[0]))


 Alice sends Bob - x value of point 

decimal: 47,457,533,883,064,335,305,529,293,257,332,224,217,655,938,374,230,319,054,306,117,409,053,034,964,544 

number of digits: 77 

hex: 68ec0325c1541731482111e4e65c09e1e6c14efc2742aa349e102f85fdd86640 

number of bits: 255 



In [14]:
my_print_number("Alice sends Bob - y value of point", Integer(alice_sends[1]))


 Alice sends Bob - y value of point 

decimal: 77,125,857,693,154,141,988,445,172,794,717,258,876,729,484,426,912,370,667,727,820,542,322,069,862,013 

number of digits: 77 

hex: aa83b041532cb9aab85941147aa879e338d2f0f1e6afe2ad083aab541801f67d 

number of bits: 256 



#### Likewise, Bob generates a scalar multiplier, calculates bob_scalar * P (mod p) and sends this point to Alice. Bob of course, keeps his scalar multiplier secret!

In [15]:
bob_scalar = my_random_k(p)

In [16]:
my_print_number("Bob's secret scalar multiplier", bob_scalar)


 Bob's secret scalar multiplier 

decimal: 72,575,371,214,636,780,676,415,823,585,745,612,619,915,041,628,022,545,509,300,674,356,892,792,098,656 

number of digits: 77 

hex: a0743454fa3fc91f3a37477cf436d061c729be24eeaadcbd2fc84907e5058f60 

number of bits: 256 



In [17]:
bob_sends = bob_scalar * P

In [18]:
my_print_number("Bob sends Alice - x value of point", Integer(bob_sends[0]))


 Bob sends Alice - x value of point 

decimal: 25,695,541,418,128,083,452,113,344,738,415,397,277,515,182,460,439,831,130,231,715,561,311,182,419,796 

number of digits: 77 

hex: 38cf293abbcfe10bfb48261d2c64f658dc6aa044a8a38700e5b82d5637867754 

number of bits: 254 



In [19]:
my_print_number("Bob sends Alice - y value of point", Integer(bob_sends[1]))


 Bob sends Alice - y value of point 

decimal: 1,380,775,109,021,823,308,990,774,472,440,304,762,898,477,151,526,546,477,878,457,812,209,591,676,213 

number of digits: 76 

hex: 30d7db34560270053f28c68c9dd45dc8306c0422aebf94ed47e958329a46d35 

number of bits: 250 



#### Both Alice and Bob should validate that the point they received from the other is indeed a valid point on the same elliptic curve to guard against an invalid curve attack!

In [20]:
alice_sends.curve()

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Finite Field of size 115792089210356248762697446949407573530086143415290314195533631308867097853951

In [21]:
bob_sends.curve()

Elliptic Curve defined by y^2 = x^3 + 115792089210356248762697446949407573530086143415290314195533631308867097853948*x + 41058363725152142129326129780047268409114441015993725554835256314039467401291 over Finite Field of size 115792089210356248762697446949407573530086143415290314195533631308867097853951

In [22]:
if alice_sends.curve() == bob_sends.curve():
    print ("Both Alice's point and Bob's point are valid on the same curve")
else:
    print ("Warning - invalid curve attack!")

Both Alice's point and Bob's point are valid on the same curve


#### Alice and Bob can now both calculate the key (which should be the same for both).  Note that neither picked the key.  Note also that the key is a point on the elliptic curve (two large values x and y of the point)

In [23]:
alice_key = alice_scalar * bob_sends

In [24]:
my_print_number("Alice's key - x value of point", Integer(alice_key[0]))


 Alice's key - x value of point 

decimal: 37,699,830,567,399,042,344,558,376,799,519,049,888,997,197,760,677,190,580,306,811,491,944,660,655,672 

number of digits: 77 

hex: 53595912346757830d3ef04b2493ed95078a3e9857008e4e69a43789499ede38 

number of bits: 255 



In [25]:
my_print_number("Alice's key - y value of point", Integer(alice_key[1]))


 Alice's key - y value of point 

decimal: 31,073,877,323,866,132,679,333,296,437,863,445,848,419,295,787,719,844,847,699,806,682,613,224,563,449 

number of digits: 77 

hex: 44b330f8f43337369898a74e88fb1267f7c04338a6edc1579019651cd527e2f9 

number of bits: 255 



In [26]:
bob_key = bob_scalar * alice_sends

In [27]:
my_print_number("Bob's key - x value of point", Integer(bob_key[0]))


 Bob's key - x value of point 

decimal: 37,699,830,567,399,042,344,558,376,799,519,049,888,997,197,760,677,190,580,306,811,491,944,660,655,672 

number of digits: 77 

hex: 53595912346757830d3ef04b2493ed95078a3e9857008e4e69a43789499ede38 

number of bits: 255 



In [28]:
my_print_number("Bob's key - y value of point", Integer(bob_key[1]))


 Bob's key - y value of point 

decimal: 31,073,877,323,866,132,679,333,296,437,863,445,848,419,295,787,719,844,847,699,806,682,613,224,563,449 

number of digits: 77 

hex: 44b330f8f43337369898a74e88fb1267f7c04338a6edc1579019651cd527e2f9 

number of bits: 255 



#### Validate that Alice's key and Bob's key match

In [29]:
if alice_key == bob_key:
    print ("Alice's key matches Bob's key")
else:
    print ("Alice's key does NOT match Bob's key")

Alice's key matches Bob's key


#### Since the generated key is a point on an elliptic curve, and we typically want to use it as a key for AES256 symmetrical encryption with a 256 bit key, it's common to concatenate the x and y values of the point and take a SHA256 hash.

In [30]:
alice_sha256 = hashlib.sha256((str(alice_key[0]) + str(alice_key[1])).encode('utf-8')).hexdigest()

bob_sha256 = hashlib.sha256((str(bob_key[0]) + str(bob_key[1])).encode('utf-8')).hexdigest()

print ("\n Alice and Bob should both have the same sha256 values for the key generated by Diffie Hellman Elliptic Curve:")

print ("Alice's sha256: ",  alice_sha256)
print ("  Bob's sha256: ", bob_sha256)


 Alice and Bob should both have the same sha256 values for the key generated by Diffie Hellman Elliptic Curve:
Alice's sha256:  b19d41cd48de1cea2c2538d2b9ceec312511ae530cbf1a5baab2b4f9e8676547
  Bob's sha256:  b19d41cd48de1cea2c2538d2b9ceec312511ae530cbf1a5baab2b4f9e8676547
